In [2]:
import requests
from pydub import AudioSegment
import os

In [3]:
# VoicevoxエンジンのURL
BASE_URL = "http://127.0.0.1:50021"


def synthesize_voice(text, speaker, output_file):
    """
    Voicevoxを使って音声を生成し、指定したファイルに保存
    :param text: 合成するテキスト
    :param speaker: スピーカーID
    :param output_file: 保存先ファイルパス
    """
    # 音声クエリを生成
    response = requests.post(
        f"{BASE_URL}/audio_query", params={"text": text, "speaker": speaker}
    )
    if response.status_code != 200:
        print("クエリ生成に失敗:", response.text)
        return None

    query = response.json()

    # 音声を生成
    response = requests.post(
        f"{BASE_URL}/synthesis", json=query, params={"speaker": speaker}
    )
    if response.status_code != 200:
        print("音声生成に失敗:", response.text)
        return None

    # ファイルに保存
    with open(output_file, "wb") as f:
        f.write(response.content)
    print(f'File Save : {output_file}')

    return output_file

In [4]:
def generate_conversation(conversation_script, output_file="wav/conversation.wav"):
    """
    会話のスクリプトを基に音声を生成し、1つの音声ファイルにまとめる
    :param conversation_script: 会話スクリプト (リスト形式)
    :param output_file: 保存する最終的な音声ファイル
    """
    # 各セリフの音声を生成
    audio_segments = []
    for i, line in enumerate(conversation_script):
        text = line["text"]
        speaker = line["speaker"]
        tmp_file = f"../wav/tmp_{i+1:02}.wav"
        synthesize_voice(text, speaker, tmp_file)
        segment = AudioSegment.from_file(tmp_file)
        audio_segments.append(segment)
        os.remove(tmp_file)
        

    # 全ての音声を結合
    # combined_audio = sum(audio_segments)
    silence = AudioSegment.silent(duration=500)  # 0.5秒の無音
    combined_audio = sum([segment + silence for segment in audio_segments])

    # ファイルに保存
    combined_audio.export(output_file, format="wav")
    print(f"会話音声を保存しました: {output_file}")

In [5]:
def conversation_script(scripts, speaker1, speaker2):
    '''テキストとスピーカで辞書を作成'''
    conversation_script = []
    for i, text in enumerate(scripts.split()):
        speaker = speaker1
        if i % 2 == 1:
            speaker=speaker2
        tmp = {'text': text, 'speaker': speaker}
        conversation_script.append(tmp)
    return conversation_script

In [6]:
# 実行例
if __name__ == "__main__":

    scripts = '''
    こんにちは、お元気ですか？
    疲れているようですね。
    昨晩はよく眠れましたか？
    いいえ、眠れませんでした。
    どうして？ 昨日は何をしましたか？
    昨晩、ナイトクラブに行って一晩中踊りました。
    ああ、そうなんですか？ 楽しかったですか？
    素晴らしい時間を過ごしましたが、今日は疲れ切っています。
    何時にナイトクラブを出ましたか？
    午前3時ごろに出ました。
    疲れているのも無理はないですね。
    どのナイトクラブに行ったんですか？
    ファンタスティックです。オンタリオ・ストリートにあります。とても素敵なところです。
    '''
    
    speaker1 = 29
    speaker2 = 58

    # # 重複しないランダムな数字を取得
    # SPEAKERS_LIST = [
    #     2, 3, 8, 10, 9, 11, 13, 14, 16, 20, 21, 23, 29, 42, 43, 46, 47, 51, 52, 53, 54, 55, 58, 61, 67, 68, 69, 74, 89
    #     ]
    # speaker1, speaker2 = random.sample(SPEAKERS_LIST, 2)

    output_filename = "../wav/conversation.wav"

    conversation_script = conversation_script(scripts, speaker1, speaker2)
    
    generate_conversation(conversation_script, output_filename)

File Save : ../wav/tmp_01.wav
File Save : ../wav/tmp_02.wav
File Save : ../wav/tmp_03.wav
File Save : ../wav/tmp_04.wav
File Save : ../wav/tmp_05.wav
File Save : ../wav/tmp_06.wav
File Save : ../wav/tmp_07.wav
File Save : ../wav/tmp_08.wav
File Save : ../wav/tmp_09.wav
File Save : ../wav/tmp_10.wav
File Save : ../wav/tmp_11.wav
File Save : ../wav/tmp_12.wav
File Save : ../wav/tmp_13.wav
File Save : ../wav/tmp_14.wav
File Save : ../wav/tmp_15.wav
会話音声を保存しました: ../wav/conversation.wav
